# 02 - Energy Analysis - Part 2
## Model training and evaluation

In [ ]:
# import libraries
import os
import numpy as np
import pandas as pd
import math
from scipy import stats
import sklearn.metrics as sklm
import sklearn.model_selection as ms
from sklearn import preprocessing
import joblib
import json
from sklearn.model_selection import (
    GridSearchCV,
    StratifiedKFold,
    KFold,
    cross_val_score,
    cross_val_predict,
)
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import xgboost as xgb
import colorlover as cl

# configs
pd.options.display.float_format = "{:,.2f}".format
%matplotlib inline
plt.rcParams["figure.figsize"] = 10, 6
pio.templates.default = "plotly_white"

In [ ]:
# import custom functions
from myLib import data_analysis

In [ ]:
os.listdir("./data/processed")

In [ ]:
data_bl = pd.read_csv("./data/processed/processed_data_bl.csv", index_col=0)
data_bl.head()

In [ ]:
data_bl.info()

In [ ]:
data_bl.ProdDate = data_bl.ProdDate.astype("datetime64[ns]")

In [ ]:
data_bl.columns

In [ ]:
feat_cols = ["Feed"]
print(feat_cols)

feat_labs_cols = feat_cols + ["Actual_kWh"]
print(feat_labs_cols)

In [ ]:
# split features into numpy array
features = np.array(data_bl[feat_cols])
print(features.shape)

In [ ]:
p = features.shape[1]
print(p)

In [ ]:
# split label into numpy array
label = np.array(data_bl["Actual_kWh"])

In [ ]:
# train-test split
np.random.seed(9988)

indx = range(features.shape[0])
indx = ms.train_test_split(
    indx,
    test_size=0.30,
)

x_train = features[indx[0], :]
y_train = np.ravel(label[indx[0]])
x_test = features[indx[1], :]
y_test = np.ravel(label[indx[1]])

In [ ]:
# scale features
scaler = preprocessing.StandardScaler().fit(x_train[:, :])
x_train[:, :] = scaler.transform(x_train[:, :])
x_test[:, :] = scaler.transform(x_test[:, :])

# save scaler model
joblib.dump(scaler, "./models/model_feature_scaling.pkl")

x_train[:5, :]

## Linear model
Use as base reference

In [ ]:
x_train_constant = sm.add_constant(x_train)
x_test_constant = sm.add_constant(x_test)

In [ ]:
model_lin = sm.OLS(y_train, x_train_constant)
model_lin_fit = model_lin.fit()
model_lin_fit.summary()

In [ ]:
y_hat = model_lin_fit.predict(x_train_constant)

In [ ]:
data_analysis.regression_metrics(y_train, y_hat, p)

In [ ]:
data_analysis.diagnostic_plots(x_train, y_train, y_hat)

In [ ]:
y_score = model_lin_fit.predict(x_test_constant)
data_analysis.regression_metrics(y_test, y_score, p)

In [ ]:
data_analysis.diagnostic_plots(x_test, y_test, y_score)

- Regression metrics look good with a slight curve

## XGBoost model
- Try XGBoost as it should fit the data better and is more robust against outliers

In [ ]:
# set up parameter grid
xgbm_param_grid = {
    "learning_rate": [0.01, 0.1, 0.3, 0.6, 0.9],
    "n_estimators": [25, 50, 100, 150],
    "subsample": [0.3, 0.5, 0.9],
    "colsample_bytree": [0.3, 0.5, 0.7],
    "gamma": [0.3, 0.5, 0.7],
    "max_depth": [3, 5, 7, 9],
    "objective": ["reg:squarederror"],
}

In [ ]:
k_fold = KFold(n_splits=3)

In [ ]:
model_xgb = xgb.XGBRegressor()

In [ ]:
grid_mse = GridSearchCV(
    estimator=model_xgb,
    param_grid=xgbm_param_grid,
    scoring="neg_mean_squared_error",
    cv=k_fold,
    verbose=1,
    n_jobs=8,
)

In [ ]:
grid_mse.fit(x_train, y_train)

In [ ]:
# print the best parameters and lowest RMSE
print(f"Best parameters found: {grid_mse.best_params_}")
print(f"Lowest RMSE found: {np.sqrt(np.abs(grid_mse.best_score_)):0.2f}")

In [ ]:
# use best parameters
model_xgb = xgb.XGBRegressor(
    colsample_bytree=0.3,
    gamma=0.3,
    learning_rate=0.1,
    max_depth=3,
    n_estimators=50,
    subsample=0.3,
)

In [ ]:
# fit model
model_xgb_fit = model_xgb.fit(x_train, y_train)

In [ ]:
y_hat = model_xgb_fit.predict(x_train)
data_analysis.regression_metrics(y_train, y_hat, p)

In [ ]:
data_analysis.diagnostic_plots(x_train, y_train, y_hat)

In [ ]:
y_score = model_xgb_fit.predict(x_test)
data_analysis.regression_metrics(y_test, y_score, p)

In [ ]:
data_analysis.diagnostic_plots(x_test, y_test, y_score)

- Regression statistics look very similar, however model took out most of the non-linearity

In [ ]:
# save model model
joblib.dump(model_xgb_fit, "./models/model_predict_xgboost.pkl")

In [ ]:
SEmodel = math.sqrt(sklm.mean_squared_error(y_test, y_score))
print(f"Standard error of the model is {SEmodel:0.2f}")

## Calculate energy performance using the baseline model

In [ ]:
os.listdir("./data/interim")

In [ ]:
# load reporting period data
data_rp = pd.read_csv("./data/interim/interim_data_rp.csv", index_col=0)
# apply filters similar to baseline data
data_rp = data_rp[data_rp.ProdDate < "2020-12-01"]
data_rp = data_rp[data_rp.Feed > 200]
data_rp = data_rp[data_rp.Actual_kWh > 55000]
data_rp.head()

In [ ]:
data_rp.info()

In [ ]:
data_rp.ProdDate = data_rp.ProdDate.astype("datetime64[ns]")

In [ ]:
data_rp.columns

In [ ]:
scl = cl.scales["9"]["seq"]["Blues"]
colorscale = [[float(i) / float(len(scl) - 1), scl[i]] for i in range(len(scl))]

trace = go.Scatter(
    x=data_rp.Feed,
    y=data_rp.Actual_kWh,
    text=data_rp.index,
    mode="markers+text",
    textposition="top center",
    hoverinfo="text",
    marker=dict(opacity=0.5, sizemin=5, sizemode="area"),
)
trace_c = go.Histogram2dContour(
    x=data_rp.Feed,
    y=data_rp.Actual_kWh,
    ncontours=5,
    colorscale=colorscale,
    showscale=False,
    opacity=0.3,
)
data = [trace, trace_c]
layout = go.Layout(title="Scatter plot")
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
data_rp.loc[
    [
        690,
        364,
        367,
        578,
        368,
        302,
        532,
        582,
        581,
        580,
        533,
        450,
        445,
        654,
        656,
        449,
        332,
    ]
].sort_values(by="ProdDate")

In [ ]:
data_rp.drop(
    [
        690,
        364,
        367,
        578,
        368,
        302,
        532,
        582,
        581,
        580,
        533,
        450,
        445,
        654,
        656,
        449,
        332,
    ],
    inplace=True,
)

In [ ]:
sns.lmplot(
    data=data_rp,
    x="Feed",
    y="Actual_kWh",
    lowess=True,
    line_kws={"color": "red"},
    aspect=1.8,
)
sns.kdeplot(x=data_rp.Feed, y=data_rp.Actual_kWh)
plt.ylabel("Energy consumption [kWh]")
plt.xlabel("Feed to plant [tons]")
plt.show()

In [ ]:
# split features into numpy array
features_rp = np.array(data_rp[feat_cols])
print(features_rp.shape)

In [ ]:
os.listdir("./models")

In [ ]:
# import models for reporting period
scaler = joblib.load("./models/model_feature_scaling.pkl")
model_opt = joblib.load("./models/model_predict_xgboost.pkl")

In [ ]:
x_rp = scaler.transform(features_rp[:, :])
x_rp[:5, :]

In [ ]:
# predict expected consumption
y_rp = model_opt.predict(x_rp)

- Calculate the CUSUM = actual minus expected energy consumption
- Compare against a target of 3% improvement

In [ ]:
data_rp["ExpectedkWh"] = y_rp

data_rp["Residuals"] = data_rp.Actual_kWh - data_rp.ExpectedkWh
data_rp["CUSUM"] = data_rp.Residuals.cumsum()

data_rp["TargetkWh"] = data_rp.ExpectedkWh * 0.97
data_rp["ResidualsT"] = data_rp.TargetkWh - data_rp.ExpectedkWh
data_rp["CUSUMT"] = data_rp.ResidualsT.cumsum()

print(
    "Cumulative performance against actual: {0:,.0f} kWh".format(
        data_rp["CUSUM"].tail(1).values[0]
    )
)

In [ ]:
traceE = go.Scatter(
    x=data_rp.ProdDate, y=data_rp.CUSUM, name="Cumulative energy performance [kWh]"
)
traceT = go.Scatter(x=data_rp.ProdDate, y=data_rp.CUSUMT, name="3% Target [kWh]")

layout = go.Layout(
    legend=dict(orientation="h"),
    title="Cumulative energy performance",
    xaxis=dict(
        title="",
        titlefont=dict(
            # size=18,
            color="#7f7f7f"
        ),
    ),
    yaxis=dict(
        title="Cumulative energy [kWh]",
        titlefont=dict(
            # size=18,
            color="#7f7f7f"
        ),
    ),
)
data = [traceE, traceT]
fig = go.Figure(data=data, layout=layout)
fig.show()

- Tracking energy performance shows about a 3% improvement up until May 2020 (downward trend)
- However, from May 2020, about 6% more energy was consumed than expected
- After following up with the plant, from around May 2020, more electrical equipment was added that is not related to plant processing, but is fed from the same electrical supply
- The baseline model needs to be revisited using the second half of 2020 as the baseline and additional relevant variables incorporated

In [ ]:
data_rp.to_csv("./data/processed/processed_data_rp.csv")